In [3]:
from Bio import Entrez
from Bio import SeqIO

EMAIL = 's2614533@ed.ac.uk'
Entrez.email = EMAIL

In [ ]:
def get_content(sequence, nucleotide):
    return round(100 * sequence.count(nucleotide)/len(sequence),2)

def get_most_frequent_aminoacid(sequence):
    aminoacids = ['A','R','D','N','C','E','Q','G','H','I','L','K','M','F','P','S','T','W','Y','V']
    
    max_count = 0
    most_frequent_aminoacids = []

    for a in aminoacids:
        count = sequence.count(a)
        if count > max_count:
            most_frequent_aminoacids = [a]
            max_count = count
        elif count == max_count:
            most_frequent_aminoacids.append(a)
    
    return most_frequent_aminoacids

    
accession_ids = 'NM_033646.4, NM_004361.5, NM_001317214.3, NM_001362438.2'
handle = Entrez.efetch( db = 'Nucleotide', id = accession_ids, rettype = 'gb', retmode = 'text')
records = list(SeqIO.parse(handle, 'genbank')) # SeqIO.parse returns an iterator which can be used to iterate only once. Since we want to iterate twice, we need to use a list

print(f"{'Accession number':20}{'%G':10}{'%C':10}{'%T':10}{'%A':10}{'Length':10}")
for entry in records:
    sequence = entry.seq
    print(f"{entry.id:20}{get_content(sequence, 'G'):<10}{get_content(sequence, 'C'):<10}{get_content(sequence, 'T'):<10}{get_content(sequence, 'A'):<10}{len(sequence):<10}")


print("\n\n")
# Do we need to find translations to protein in all 6 reading frames?
for entry in records:
    print(f"Accession ID of the gene transcript: {entry.id}")
    for feature in entry.features:
        if feature.type == "CDS":
            print(f"Protein ID: {feature.qualifiers['protein_id']}")
            print(f"Location of the CDS = {feature.location}\n")
            current_sequence = feature.location.extract(entry).seq
            print('Protein Sequence')
            protein_sequence = current_sequence.translate(to_stop = True)
            print(protein_sequence)
            print(f"Length of the protein sequence = {len(protein_sequence)}")
            print(f"Most frequent aminoacid(s): {get_most_frequent_aminoacid(protein_sequence)} \n")


In [ ]:
handle = Entrez.efetch(db="gene", id='1005', rettype="gb", retmode="text")
print(handle.read())


By searching the NCBI databases through NCBI website, we have found this information about human Calderin 7: 
Gene ID: 1005
Official Symbol: CDH7
Also known as: CDH7L1

https://www.ncbi.nlm.nih.gov/nuccore/NM_004361.5,NM_033646.4,NM_001317214.3,NM_001362438.2
4 different transcripts:
a) NM_033646.4      12126 bp
b) NM_004361.5      12136 bp
c) NM_001317214.3   3407 bp
d) NM_001362438.2   12938 bp 

In [3]:
# pairwise sequence alignment
from Bio import Entrez
from Bio import SeqIO
from Bio import pairwise2 as pw
from Bio import AlignIO
from Bio import Align as al

EMAIL = 's2614533@ed.ac.uk'
Entrez.email = EMAIL

mx = al.substitution_matrices.load('BLOSUM90')

shortest_seq = records[2].seq
longest_seq = records[3].seq
alignments = pw.align.localds(longest_seq,shortest_seq,mx, -10, -0.5)

In [ ]:
print("Number of different alignments: ",len(alignments))
print("Score: ",alignments[0][2])

# print(pw.format_alignment(*alignments[0]))

alignment_fasta = \
">"+records[3].name+" "+records[3].description+"\n"+alignments[0][0] \
+"\n"+ \
">"+records[2].name+" "+records[2].description+"\n"+alignments[0][1]

# write it to a file
fh = open('calderin_alignment_blosum62.fa','w')
fh.write(alignment_fasta)
fh.close()

# read in the file using AlignIO
alignment = AlignIO.read('calderin_alignment_blosum62.fa', "fasta")

# convert to clustal
print(format(alignment,'clustal'))

also used megablast to compare the sequences:
https://blast.ncbi.nlm.nih.gov/Blast.cgi

why megablast - for highly similar sequences
why BLOSUM90 ?? - explain
why local search - ??

In [9]:
for entry in records:
    count = 1
    lengths = []
    locations = []
    for feature in entry.features:
        if feature.type == "exon":
            # print(f"Location of exon {count} = {feature.location}")
            exon = feature.location.extract(entry).seq
            # print(f"length = {len(exon)}")
            #count += 1
            lengths.append(len(exon))
            locations.append(str(feature.location))
    print(locations)
    print(lengths)
    
        

['[0:129](+)', '[129:535](+)', '[535:830](+)', '[830:950](+)', '[950:1118](+)', '[1118:1306](+)', '[1306:1560](+)', '[1560:1697](+)', '[1697:1819](+)', '[1819:1937](+)', '[1937:2189](+)', '[2189:12126](+)']
[129, 406, 295, 120, 168, 188, 254, 137, 122, 118, 252, 9937]
['[0:139](+)', '[139:545](+)', '[545:840](+)', '[840:960](+)', '[960:1128](+)', '[1128:1316](+)', '[1316:1570](+)', '[1570:1707](+)', '[1707:1829](+)', '[1829:1947](+)', '[1947:2199](+)', '[2199:12136](+)']
[139, 406, 295, 120, 168, 188, 254, 137, 122, 118, 252, 9937]
['[0:139](+)', '[139:545](+)', '[545:840](+)', '[840:960](+)', '[960:1128](+)', '[1128:1316](+)', '[1316:1570](+)', '[1570:1707](+)', '[1707:1829](+)', '[1829:1947](+)', '[1947:3407](+)']
[139, 406, 295, 120, 168, 188, 254, 137, 122, 118, 1460]
['[0:941](+)', '[941:1347](+)', '[1347:1642](+)', '[1642:1762](+)', '[1762:1930](+)', '[1930:2118](+)', '[2118:2372](+)', '[2372:2509](+)', '[2509:2631](+)', '[2631:2749](+)', '[2749:3001](+)', '[3001:12938](+)']
[941

TASK 3
13 exons in total (https://www.ncbi.nlm.nih.gov/gene/1005)
Lengths:
* exon 1 : 129 (present only in variant a)
* exon 2a: 139
* exon 2b: 941
* exon 3: 406
* exon 4: 295
* exon 5: 120
* exon 6: 168
* exon 7: 188
* exon 8: 254
* exon 9: 137
* exon 10: 122
* exon 11: 118
* exon 12a: 252
* exon 12b: 1460
* exon 13: 9937


shortest transcript has 11 exons, longest transcript has 12 exons
Exons 2 - 10 are identical (seem so, check!)
Exon 1 in the shortest transcript is only part of the exon 1 in the longest transcript
Exon 11 in the longest transcript is only part of the exon 11 in the shortest transcript

Visualisation:
https://www.ncbi.nlm.nih.gov/genome/gdv/browser/gene/?id=1005